In [1]:
import tensorflow as tf
import os
import _pickle as cPickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"

from myUtils.utils import CifarData

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

batch_data, batch_labels = train_data.next_batch(10)
print(batch_data)
print(batch_labels)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)
[[-0.34901961 -0.36470588 -0.36470588 ... -0.27843137 -0.30196078
  -0.38039216]
 [ 0.30196078  0.27843137  0.27843137 ...  0.61568627  0.61568627
   0.6       ]
 [ 0.97647059  0.96078431  0.96862745 ...  0.86666667  0.92156863
   0.9372549 ]
 ...
 [ 0.54509804  0.52156863  0.37254902 ...  0.54509804  0.51372549
   0.4745098 ]
 [-0.74117647 -0.44313725 -0.21568627 ... -0.57647059 -0.45882353
  -0.28627451]
 [ 0.03529412  0.10588235  0.00392157 ... -0.38823529 -0.39607843
  -0.41176471]]
[8 2 2 1 7 3 6 5 1 2]


In [2]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

w = tf.get_variable('w', [x.get_shape()[-1], 10], initializer=tf.random_normal_initializer(0, 1))
b = tf.get_variable('b', [1], initializer=tf.constant_initializer(0.0))

y_ = tf.matmul(x, w) + b

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)

# p_y = tf.nn.softmax(y_)
# y_one_hot = tf.one_hot(y, 10, dtype=tf.float32)
# loss = tf.reduce_mean(tf.square(y_one_hot - p_y))

# p_y_1 = tf.nn.sigmoid(y_)
# y_reshaped = tf.reshape(y, (-1, 1))
# loss = tf.reduce_mean(tf.square(y_reshaped_float - p_y_1))

predict = tf.argmax(y_, 1) 
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, accu_val, _ = sess.run([loss, accuracy, train_op], feed_dict={x: batch_data, y: batch_labels})
        if (i + 1) % 500 == 0 :
            print("[Train] Step: %d, loss: %4.5f, accuracy: %4.5f" % (i + 1, loss_val, accu_val))
        if (i + 1) % 5000 == 0 :
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps) :
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict={x: test_batch_data, y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print("[Test] Step: %d, accuracy: %4.5f" % (i + 1, test_acc))

[Train] Step: 500, loss: 0.14900, accuracy: 0.20000
[Train] Step: 1000, loss: 0.15319, accuracy: 0.20000
[Train] Step: 1500, loss: 0.16749, accuracy: 0.15000
[Train] Step: 2000, loss: 0.16537, accuracy: 0.15000
[Train] Step: 2500, loss: 0.12829, accuracy: 0.35000
[Train] Step: 3000, loss: 0.12322, accuracy: 0.35000
[Train] Step: 3500, loss: 0.14733, accuracy: 0.25000
[Train] Step: 4000, loss: 0.13933, accuracy: 0.30000
[Train] Step: 4500, loss: 0.13009, accuracy: 0.30000
[Train] Step: 5000, loss: 0.12177, accuracy: 0.35000
(10000, 3072)
(10000,)
[Test] Step: 5000, accuracy: 0.31400
[Train] Step: 5500, loss: 0.13722, accuracy: 0.30000
[Train] Step: 6000, loss: 0.14756, accuracy: 0.25000
[Train] Step: 6500, loss: 0.13225, accuracy: 0.35000
[Train] Step: 7000, loss: 0.13573, accuracy: 0.30000
[Train] Step: 7500, loss: 0.15624, accuracy: 0.20000
[Train] Step: 8000, loss: 0.14711, accuracy: 0.25000
[Train] Step: 8500, loss: 0.11927, accuracy: 0.40000
[Train] Step: 9000, loss: 0.13389, accur

[Train] Step: 69500, loss: 0.09456, accuracy: 0.50000
[Train] Step: 70000, loss: 0.08938, accuracy: 0.55000
(10000, 3072)
(10000,)
[Test] Step: 70000, accuracy: 0.38650
[Train] Step: 70500, loss: 0.11431, accuracy: 0.40000
[Train] Step: 71000, loss: 0.08738, accuracy: 0.55000
[Train] Step: 71500, loss: 0.13268, accuracy: 0.30000
[Train] Step: 72000, loss: 0.10175, accuracy: 0.50000
[Train] Step: 72500, loss: 0.07638, accuracy: 0.55000
[Train] Step: 73000, loss: 0.10401, accuracy: 0.45000
[Train] Step: 73500, loss: 0.10584, accuracy: 0.45000
[Train] Step: 74000, loss: 0.13356, accuracy: 0.30000
[Train] Step: 74500, loss: 0.16149, accuracy: 0.15000
[Train] Step: 75000, loss: 0.10366, accuracy: 0.45000
(10000, 3072)
(10000,)
[Test] Step: 75000, accuracy: 0.38700
[Train] Step: 75500, loss: 0.13851, accuracy: 0.30000
[Train] Step: 76000, loss: 0.11857, accuracy: 0.40000
[Train] Step: 76500, loss: 0.11397, accuracy: 0.40000
[Train] Step: 77000, loss: 0.11993, accuracy: 0.40000
[Train] Step: 